In [86]:
import pandas as pd
import numpy as np
from subprocess import run
import requests
from gzip import decompress
from pathlib import Path

# Gen interest bed file

In [6]:
gene_address = pd.read_csv("ensg.csv", delimiter="\t")

In [ ]:
arr = []
for _, gene, chrom, _, _, _, start, end in gene_address.itertuples():
    if end - start != 10000:
        continue
    for x in range(start, end, 100):
        arr.append((chrom, x, x+100, gene))
    

In [ ]:
pd.DataFrame(arr).to_csv("interest.bed", sep="\t", header=False, index=False)

# Get Chip-seq Data

In [11]:
gene_expresson = pd.read_csv("57epigenomes.RPKM.pc", delimiter="\t", index_col=False)

In [81]:
epigenomes = gene_expresson.columns[2:]
histones = ["H3K27me3", "H3K26me3", "H3K4me1", "H3K4me3", "H3K9me3"]

In [ ]:
completed_records = set()

In [85]:
for epigenome in epigenomes:
    for histone in histones:
        addr = f"http://egg2.wustl.edu/roadmap/data/byFileType/alignments/consolidated/{epigenome}-{histone}.tagAlign.gz"
        target = f"data/{epigenome}-{histone}.bed"
        
        if Path(target).is_file():
            resp = requests.get(addr)
            with open(target, "wb") as f:
                f.write(decompress(resp.content))
        break
    break

http://egg2.wustl.edu/roadmap/data/byFileType/alignments/consolidated/E003-H3K27me3.tagAlign.gz <Response [200]>


TypeError: write() argument must be str, not bytes